<a href="https://colab.research.google.com/gist/maclandrol/ab9a6ec3c96162e39c65c34e75596095/07_TorchXRayVision_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Enseignant:** Emmanuel Noutahi, PhD

# Tutorial 2: Classification Avancée de Radiographies avec TorchXRayVision
## Multi-Pathologies et Comparaison de Modèles

---

## 🎯 Objectifs de ce Tutorial

Dans ce deuxième tutorial, vous allez maîtriser :

1. **Classification multi-pathologies** avec différents modèles
2. **Comparaison de performances** entre architectures
3. **Analyse de cas cliniques** complexes
4. **Métriques d'évaluation** avancées
5. **Interprétation clinique** des résultats
6. **Workflow hospitalier** intégré

---

## 🏥 Prérequis

Ce tutorial fait suite au **Tutorial 1**. Assurez-vous d'avoir :
- ✅ Terminé le Tutorial 1 (Introduction)
- ✅ TorchXRayVision installé et configuré
- ✅ Compréhension du preprocessing
- ✅ Bases de l'interprétation IA

---

## 🔬 Introduction Médicale Avancée

### Défis de la Classification Multi-Pathologies

En pratique clinique réelle, les radiographies présentent souvent :
- **Pathologies multiples simultanées**
- **Degrés de sévérité variables**
- **Présentations atypiques**
- **Artéfacts et limitations techniques**

### 📊 Enjeux Cliniques

#### **Sensibilité vs Spécificité :**
- **Sensibilité élevée** : Ne pas manquer de pathologies (faux négatifs)
- **Spécificité élevée** : Éviter les fausses alertes (faux positifs)
- **Trade-off clinique** selon le contexte (urgences vs dépistage)

#### **Impact des Modèles :**
- **Modèles généralistes** : Performance globale équilibrée
- **Modèles spécialisés** : Excellence sur pathologies ciblées
- **Ensembles de modèles** : Robustesse et fiabilité accrues

---

## 🔧 Configuration Avancée

In [ ]:
# Configuration avancée pour classification multi-modèles
import sys
import torch
import warnings
warnings.filterwarnings('ignore')

print("🏥 TUTORIAL 2: CLASSIFICATION AVANCÉE TORCHXRAYVISION")
print("=" * 65)

# Vérifications système
print(f"\n🔧 Configuration système :")
if 'google.colab' in sys.modules:
    print("   • Environnement : Google Colab ✅")
    IN_COLAB = True
else:
    print("   • Environnement : Local")
    IN_COLAB = False

# Configuration GPU optimisée
if torch.cuda.is_available():
    device = "cuda"
    print(f"   • GPU : {torch.cuda.get_device_name(0)} ✅")
    print(f"   • Mémoire : {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    # Configuration CUDA optimisée
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    print(f"   • CUDNN optimisé : ✅")
else:
    device = "cpu"
    print(f"   • GPU : Non disponible")
    print(f"   • Mode : CPU (plus lent pour multi-modèles)")

# Configuration pour reproducibilité
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

print(f"\n🎯 Dispositif sélectionné : {device}")
print(f"🔄 Reproductibilité : Activée (seed=42)")

print("\n✅ Configuration avancée terminée !")

## 📦 Importations et Vérifications

In [ ]:
# Importations complètes pour classification avancée
print("📦 IMPORTATION DES BIBLIOTHÈQUES AVANCÉES")
print("=" * 45)

try:
    # Core libraries
    import torchxrayvision as xrv
    import torch
    import torchvision
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from PIL import Image
    import skimage
    
    # Advanced analysis
    from sklearn.metrics import (
        roc_curve, auc, confusion_matrix, 
        classification_report, precision_recall_curve
    )
    from scipy import stats, ndimage
    from datetime import datetime
    import os
    import json
    import time
    
    print("✅ Bibliothèques principales importées")
    print("✅ Métriques d'évaluation chargées")
    print("✅ Outils d'analyse statistique prêts")
    
except ImportError as e:
    print(f"❌ Erreur d'importation : {e}")
    print("💡 Installez les dépendances manquantes")

# Configuration matplotlib avancée
plt.style.use('default')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Palette médicale professionnelle
medical_colors = {
    'positive': '#FF4444',    # Rouge pour pathologies
    'negative': '#44FF44',    # Vert pour normal
    'uncertainty': '#FFAA44', # Orange pour incertain
    'neutral': '#4444FF',     # Bleu pour neutre
    'background': '#F0F0F0'   # Gris clair pour fond
}

print("\n🎨 Configuration d'affichage médical appliquée")
print(f"📊 Version TorchXRayVision : {xrv.__version__}")

# Configuration de session
if IN_COLAB:
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        base_dir = '/content/drive/MyDrive/TorchXRayVision_Tutorials/'
        tutorial_dir = f"{base_dir}Tutorial_2_Classification/"
        session_dir = f"{tutorial_dir}Session_{datetime.now().strftime('%Y%m%d_%H%M%S')}/"
        os.makedirs(session_dir, exist_ok=True)
        print(f"📁 Drive monté : {session_dir}")
    except:
        session_dir = './tutorial_2_results/'
        os.makedirs(session_dir, exist_ok=True)
else:
    session_dir = './tutorial_2_results/'
    os.makedirs(session_dir, exist_ok=True)
    print(f"📁 Dossier local : {session_dir}")

print("\n🚀 Prêt pour classification avancée !")

## 🧠 Chargement de Modèles Multiples

Chargeons différents modèles pour comparaison de performances :

In [ ]:
print("🧠 CHARGEMENT DE MODÈLES MULTIPLES")
print("=" * 38)

def load_multiple_models(device, detailed_info=True):
    """
    Charge plusieurs modèles TorchXRayVision pour comparaison
    
    Cette approche multi-modèles permet :
    - Comparaison de performances
    - Validation croisée des résultats
    - Analyse de consensus
    - Robustesse diagnostique
    """
    
    models = {}
    model_info = {}
    
    if detailed_info:
        print("\n📚 Modèles à charger :")
        print("   1️⃣ Modèle Universel (tous datasets)")
        print("   2️⃣ Modèle CheXpert (Stanford)")
        print("   3️⃣ Modèle NIH (National Institutes of Health)")
        print("   4️⃣ Modèle MIMIC-CXR (MIT)")
    
    # 1. Modèle universel (entraîné sur tous les datasets)
    print("\n🔄 Chargement du modèle universel...")
    try:
        models['universel'] = xrv.models.DenseNet(weights="densenet121-res224-all")
        models['universel'].eval().to(device)
        
        model_info['universel'] = {
            'name': 'Modèle Universel',
            'architecture': 'DenseNet121',
            'training_data': 'Tous datasets combinés',
            'specialty': 'Généraliste',
            'params': sum(p.numel() for p in models['universel'].parameters())
        }
        print("   ✅ Modèle universel chargé")
    except Exception as e:
        print(f"   ❌ Erreur modèle universel : {e}")
    
    # 2. Modèle CheXpert (Stanford)
    print("\n🎓 Chargement du modèle CheXpert...")
    try:
        models['chexpert'] = xrv.models.DenseNet(weights="densenet121-res224-chex")
        models['chexpert'].eval().to(device)
        
        model_info['chexpert'] = {
            'name': 'CheXpert (Stanford)',
            'architecture': 'DenseNet121',
            'training_data': 'CheXpert dataset (224k images)',
            'specialty': 'Pathologies thoraciques variées',
            'params': sum(p.numel() for p in models['chexpert'].parameters())
        }
        print("   ✅ Modèle CheXpert chargé")
    except Exception as e:
        print(f"   ❌ Erreur modèle CheXpert : {e}")
    
    # 3. Modèle NIH
    print("\n🏛️ Chargement du modèle NIH...")
    try:
        models['nih'] = xrv.models.DenseNet(weights="densenet121-res224-nih")
        models['nih'].eval().to(device)
        
        model_info['nih'] = {
            'name': 'NIH ChestX-ray',
            'architecture': 'DenseNet121',
            'training_data': 'NIH ChestX-ray8 (112k images)',
            'specialty': '14 pathologies thoraciques',
            'params': sum(p.numel() for p in models['nih'].parameters())
        }
        print("   ✅ Modèle NIH chargé")
    except Exception as e:
        print(f"   ❌ Erreur modèle NIH : {e}")
    
    # 4. Modèle MIMIC (si disponible)
    print("\n🏥 Tentative chargement modèle MIMIC...")
    try:
        models['mimic'] = xrv.models.DenseNet(weights="densenet121-res224-mimic_ch")
        models['mimic'].eval().to(device)
        
        model_info['mimic'] = {
            'name': 'MIMIC-CXR (MIT)',
            'architecture': 'DenseNet121',
            'training_data': 'MIMIC-CXR (377k images)',
            'specialty': 'Données hospitalières réelles',
            'params': sum(p.numel() for p in models['mimic'].parameters())
        }
        print("   ✅ Modèle MIMIC chargé")
    except Exception as e:
        print(f"   ⚠️ Modèle MIMIC non disponible : {e}")
    
    if detailed_info:
        print(f"\n📊 Résumé des modèles chargés :")
        print(f"   • Nombre total : {len(models)}")
        
        for model_key, info in model_info.items():
            if model_key in models:
                params_millions = info['params'] / 1e6
                print(f"   • {info['name']}: {params_millions:.1f}M paramètres")
    
    return models, model_info

# Chargement des modèles
print("🚀 Démarrage du chargement multi-modèles...")
start_time = time.time()

models_dict, models_info = load_multiple_models(device, detailed_info=True)

loading_time = time.time() - start_time
print(f"\n⏱️ Temps de chargement : {loading_time:.2f} secondes")
print(f"✅ {len(models_dict)} modèles prêts pour analyse comparative !")

# Vérification des pathologies communes
if models_dict:
    first_model = list(models_dict.values())[0]
    pathologies = first_model.pathologies
    print(f"\n🩺 Pathologies analysables : {len(pathologies)}")
    
    # Groupement par catégories cliniques
    infectious = [p for p in pathologies if any(term in p.lower() for term in ['pneumonia', 'tuberculosis'])]
    structural = [p for p in pathologies if any(term in p.lower() for term in ['pneumothorax', 'cardiomegaly', 'effusion'])]
    
    print(f"   • Maladies infectieuses : {len(infectious)}")
    print(f"   • Anomalies structurelles : {len(structural)}")
    print(f"   • Autres pathologies : {len(pathologies) - len(infectious) - len(structural)}")

print("\n🎯 Prêt pour classification comparative !")

## 🖼️ Création de Cas Cliniques Variés

Créons plusieurs radiographies représentant différents cas cliniques :

In [ ]:
print("🖼️ CRÉATION DE CAS CLINIQUES VARIÉS")
print("=" * 38)

def create_clinical_case_xray(case_type, size=224, severity='moderate'):
    """
    Crée des radiographies représentant différents cas cliniques
    
    Cases types :
    - 'normal' : Radiographie normale
    - 'pneumonia' : Pneumonie (consolidation)
    - 'cardiomegaly' : Cardiomégalie
    - 'pneumothorax' : Pneumothorax
    - 'pleural_effusion' : Épanchement pleural
    - 'complex' : Cas complexe multi-pathologies
    """
    
    print(f"   🏗️ Création cas {case_type} (sévérité: {severity})...")
    
    # Image de base
    img = np.zeros((size, size), dtype=np.float32)
    center_x, center_y = size // 2, size // 2
    
    # Structures anatomiques de base
    
    # 1. Poumons normaux
    lung_intensity = 0.3
    
    # Poumon droit
    right_lung_x = int(0.3 * size)
    right_lung_y = int(0.4 * size)
    right_lung_h = int(0.25 * size)
    right_lung_w = int(0.12 * size)
    
    rr1, cc1 = skimage.draw.ellipse(right_lung_y, right_lung_x, 
                                   right_lung_h, right_lung_w, shape=img.shape)
    img[rr1, cc1] = lung_intensity
    
    # Poumon gauche
    left_lung_x = int(0.7 * size)
    left_lung_y = int(0.4 * size)
    left_lung_h = int(0.23 * size)
    left_lung_w = int(0.12 * size)
    
    rr2, cc2 = skimage.draw.ellipse(left_lung_y, left_lung_x,
                                   left_lung_h, left_lung_w, shape=img.shape)
    img[rr2, cc2] = lung_intensity
    
    # 2. Cœur (taille selon pathologie)
    if case_type == 'cardiomegaly':
        if severity == 'mild':
            heart_size_factor = 1.2
        elif severity == 'moderate':
            heart_size_factor = 1.5
        else:  # severe
            heart_size_factor = 1.8
    else:
        heart_size_factor = 1.0
    
    heart_w = int(0.1 * size * heart_size_factor)
    heart_h = int(0.08 * size * heart_size_factor)
    heart_x = int(0.48 * size)
    heart_y = int(0.6 * size)
    
    rr3, cc3 = skimage.draw.ellipse(heart_y, heart_x, heart_h, heart_w, shape=img.shape)
    img[rr3, cc3] = 0.6
    
    # 3. Colonne vertébrale
    spine_width = int(0.025 * size)
    spine_start = int(0.1 * size)
    spine_end = int(0.9 * size)
    
    for y in range(spine_start, spine_end):
        img[y, center_x-spine_width:center_x+spine_width] = 0.8
    
    # 4. Côtes
    for rib_level in range(8):
        rib_y = int(0.15 * size) + rib_level * int(0.07 * size)
        rib_curve = int(0.03 * size)
        
        # Côtes droites et gauches
        for x in range(int(0.05 * size), int(0.95 * size)):
            if x < center_x:
                curve_offset = int(rib_curve * np.sin(np.pi * (x - 0.05 * size) / (0.45 * size)))
            else:
                curve_offset = int(rib_curve * np.sin(np.pi * (0.95 * size - x) / (0.45 * size)))
            
            y_rib = rib_y + curve_offset
            if 0 <= y_rib < size:
                img[y_rib:y_rib+2, x:x+1] = 0.7
    
    # 5. Pathologies spécifiques
    
    if case_type == 'pneumonia':
        # Consolidation pneumonique
        pneumo_intensity = 0.75 if severity == 'mild' else 0.85
        pneumo_size = 0.08 if severity == 'mild' else 0.12
        
        pneumo_y = int(0.35 * size)
        pneumo_x = int(0.25 * size)
        pneumo_h = int(pneumo_size * size)
        pneumo_w = int(pneumo_size * 0.8 * size)
        
        rr_pneumo, cc_pneumo = skimage.draw.ellipse(pneumo_y, pneumo_x,
                                                   pneumo_h, pneumo_w, shape=img.shape)
        img[rr_pneumo, cc_pneumo] = pneumo_intensity
        
        # Bronchogramme aérique (signe caractéristique)
        for i in range(3):
            broncho_y = pneumo_y + (i-1) * 5
            broncho_x_start = pneumo_x - 10
            broncho_x_end = pneumo_x + 10
            if 0 <= broncho_y < size:
                img[broncho_y, max(0, broncho_x_start):min(size, broncho_x_end)] = 0.4
    
    elif case_type == 'pneumothorax':
        # Ligne pleurale visible
        pneumo_line_x = int(0.15 * size) if severity != 'mild' else int(0.18 * size)
        pneumo_start_y = int(0.2 * size)
        pneumo_end_y = int(0.7 * size)
        
        # Ligne de décollement pleural
        img[pneumo_start_y:pneumo_end_y, pneumo_line_x:pneumo_line_x+2] = 0.9
        
        # Zone de décollement (hyperclarté)
        img[pneumo_start_y:pneumo_end_y, int(0.05 * size):pneumo_line_x] = 0.1
        
        # Affaissement pulmonaire
        collapse_factor = 0.7 if severity == 'mild' else 0.5
        affected_lung = img[pneumo_start_y:pneumo_end_y, pneumo_line_x:int(0.5 * size)]
        img[pneumo_start_y:pneumo_end_y, pneumo_line_x:int(0.5 * size)] = affected_lung * collapse_factor
    
    elif case_type == 'pleural_effusion':
        # Épanchement pleural (opacité basale)
        effusion_height = int(0.15 * size) if severity == 'mild' else int(0.25 * size)
        effusion_y_start = int(0.8 * size) - effusion_height
        effusion_y_end = int(0.8 * size)
        
        # Côté droit (plus fréquent)
        effusion_x_start = int(0.1 * size)
        effusion_x_end = int(0.5 * size)
        
        # Opacité hydrique avec ligne de Damoiseau
        for y in range(effusion_y_start, effusion_y_end):
            # Ligne concave caractéristique
            curve_offset = int(0.05 * size * np.sin(np.pi * (y - effusion_y_start) / effusion_height))
            x_limit = effusion_x_end - curve_offset
            img[y, effusion_x_start:min(size, x_limit)] = 0.8
    
    elif case_type == 'complex':
        # Cas complexe : Cardiomégalie + épanchement + consolidation
        # Déjà cardiomégalie par le facteur 1.3
        
        # Petit épanchement
        small_effusion_height = int(0.1 * size)
        effusion_start = int(0.75 * size)
        img[effusion_start:effusion_start+small_effusion_height, int(0.1*size):int(0.4*size)] = 0.75
        
        # Petite consolidation
        small_consolidation_y = int(0.3 * size)
        small_consolidation_x = int(0.7 * size)
        rr_small, cc_small = skimage.draw.ellipse(small_consolidation_y, small_consolidation_x,
                                                 int(0.05 * size), int(0.04 * size), shape=img.shape)
        img[rr_small, cc_small] = 0.7
    
    # 6. Post-processing réaliste
    
    # Bruit radiographique
    noise = np.random.normal(0, 0.04, img.shape)
    img = img + noise
    
    # Lissage gaussien (flou radiographique)
    img = ndimage.gaussian_filter(img, sigma=0.8)
    
    # Normalisation finale
    img = np.clip(img, 0, 1)
    
    print(f"   ✅ Cas {case_type} créé avec succès")
    
    return img

# Création de la collection de cas cliniques
print("\n🏗️ Création de la bibliothèque de cas cliniques...")

clinical_cases = {
    'normal': {
        'image': create_clinical_case_xray('normal'),
        'description': 'Radiographie thoracique normale',
        'expected_findings': ['Normal'],
        'clinical_context': 'Bilan de routine, patient asymptomatique'
    },
    'pneumonie_moderee': {
        'image': create_clinical_case_xray('pneumonia', severity='moderate'),
        'description': 'Pneumonie lobaire droite modérée',
        'expected_findings': ['Pneumonia', 'Infiltration'],
        'clinical_context': 'Fièvre, toux productive, dyspnée'
    },
    'cardiomegalie': {
        'image': create_clinical_case_xray('cardiomegaly', severity='moderate'),
        'description': 'Cardiomégalie modérée',
        'expected_findings': ['Cardiomegaly', 'Enlarged Cardiomediastinum'],
        'clinical_context': 'Dyspnée d\'effort, œdèmes membres inférieurs'
    },
    'pneumothorax': {
        'image': create_clinical_case_xray('pneumothorax', severity='moderate'),
        'description': 'Pneumothorax spontané modéré',
        'expected_findings': ['Pneumothorax'],
        'clinical_context': 'Douleur thoracique brutale, dyspnée'
    },
    'epanchement_pleural': {
        'image': create_clinical_case_xray('pleural_effusion', severity='moderate'),
        'description': 'Épanchement pleural droit modéré',
        'expected_findings': ['Pleural Effusion'],
        'clinical_context': 'Dyspnée progressive, douleur pleurale'
    },
    'cas_complexe': {
        'image': create_clinical_case_xray('complex'),
        'description': 'Cas complexe multi-pathologies',
        'expected_findings': ['Cardiomegaly', 'Pleural Effusion', 'Infiltration'],
        'clinical_context': 'Insuffisance cardiaque décompensée'
    }
}

print(f"\n✅ {len(clinical_cases)} cas cliniques créés !")

# Affichage de la galerie de cas
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle("Galerie de Cas Cliniques - Tutorial 2", fontsize=18, fontweight='bold')

axes = axes.flatten()

for i, (case_name, case_data) in enumerate(clinical_cases.items()):
    if i < len(axes):
        axes[i].imshow(case_data['image'], cmap='gray')
        axes[i].set_title(f"{case_data['description']}\n({case_name})", 
                         fontweight='bold', fontsize=11)
        axes[i].axis('off')
        
        # Contexte clinique en annotation
        axes[i].text(0.02, 0.02, case_data['clinical_context'], 
                    transform=axes[i].transAxes, fontsize=9,
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
                    verticalalignment='bottom')

plt.tight_layout()
plt.show()

print("\n🎯 Cas cliniques prêts pour classification comparative !")
print("📚 Prochaine étape : Analyse multi-modèles")

## 🔬 Classification Multi-Modèles

Analysons nos cas cliniques avec tous les modèles disponibles :

In [ ]:
print("🔬 CLASSIFICATION MULTI-MODÈLES")
print("=" * 34)

def preprocess_for_models(image_array):
    """
    Preprocessing standardisé pour tous les modèles
    """
    # Normalisation
    if image_array.max() > 1:
        img_norm = xrv.datasets.normalize(image_array, 255)
    else:
        img_norm = image_array
    
    # Format canal
    img_channel = img_norm[None, ...]
    
    # Transformations TorchXRayVision
    transform = torchvision.transforms.Compose([
        xrv.datasets.XRayCenterCrop(),
        xrv.datasets.XRayResizer(224)
    ])
    
    img_transformed = transform(img_channel)
    
    # Tenseur PyTorch
    img_tensor = torch.from_numpy(img_transformed).float()
    img_batch = img_tensor.unsqueeze(0)
    
    return img_batch, img_transformed

def analyze_with_all_models(image_tensor, models_dict, pathologies):
    """
    Analyse d'une image avec tous les modèles disponibles
    """
    results = {}
    
    for model_name, model in models_dict.items():
        model.eval()
        image_tensor = image_tensor.to(device)
        
        with torch.no_grad():
            # Prédiction
            raw_outputs = model(image_tensor)
            probabilities = torch.sigmoid(raw_outputs)
            probabilities_np = probabilities.cpu().numpy().squeeze()
            
            # Stockage des résultats
            results[model_name] = {
                'probabilities': probabilities_np,
                'pathologies': pathologies,
                'positive_findings': [(pathologies[i], prob) for i, prob in enumerate(probabilities_np) if prob > 0.5]
            }
    
    return results

def create_comparative_analysis(case_results, case_info):
    """
    Crée une analyse comparative des résultats multi-modèles
    """
    print(f"\n📊 ANALYSE COMPARATIVE : {case_info['description']}")
    print(f"-" * 70)
    
    # Pathologies attendues
    expected = case_info['expected_findings']
    print(f"🎯 Pathologies attendues : {', '.join(expected)}")
    print(f"🏥 Contexte clinique : {case_info['clinical_context']}")
    
    # Analyse par modèle
    print(f"\n📋 Résultats par modèle :")
    
    all_model_results = {}
    
    for model_name, results in case_results.items():
        model_info = models_info.get(model_name, {'name': model_name})
        print(f"\n   🧠 {model_info['name']} :")
        
        positive_findings = results['positive_findings']
        
        if positive_findings:
            print(f"      🚨 Détections positives ({len(positive_findings)}) :")
            for pathology, prob in sorted(positive_findings, key=lambda x: x[1], reverse=True):
                print(f"        • {pathology}: {prob:.3f} ({prob*100:.1f}%)")
        else:
            print(f"      ✅ Aucune pathologie détectée (seuil 50%)")
        
        # Vérification des pathologies attendues
        expected_detected = []
        for exp_pathology in expected:
            for detected_pathology, prob in positive_findings:
                if exp_pathology.lower() in detected_pathology.lower() or \
                   detected_pathology.lower() in exp_pathology.lower():
                    expected_detected.append((exp_pathology, detected_pathology, prob))
        
        if expected_detected:
            print(f"      ✅ Pathologies attendues détectées : {len(expected_detected)}")
            for exp, det, prob in expected_detected:
                print(f"        ✓ {exp} → {det} ({prob:.3f})")
        else:
            print(f"      ⚠️ Pathologies attendues non détectées")
        
        # Stockage pour analyse globale
        all_model_results[model_name] = results
    
    return all_model_results

# Analyse de tous les cas avec tous les modèles
print("\n🚀 Démarrage de l'analyse comparative multi-modèles...")
print(f"📊 {len(clinical_cases)} cas × {len(models_dict)} modèles = {len(clinical_cases) * len(models_dict)} analyses")

comprehensive_results = {}

if models_dict:  # Vérifier qu'il y a au moins un modèle
    # Récupération des pathologies du premier modèle
    first_model = list(models_dict.values())[0]
    pathologies = first_model.pathologies
    
    for case_name, case_data in clinical_cases.items():
        print(f"\n🔄 Analyse du cas : {case_name}...")
        
        # Preprocessing
        processed_tensor, processed_display = preprocess_for_models(case_data['image'])
        
        # Analyse avec tous les modèles
        case_results = analyze_with_all_models(processed_tensor, models_dict, pathologies)
        
        # Analyse comparative
        comparative_analysis = create_comparative_analysis(case_results, case_data)
        
        # Stockage
        comprehensive_results[case_name] = {
            'case_info': case_data,
            'processed_image': processed_display,
            'model_results': comparative_analysis
        }
        
        print(f"   ✅ Cas {case_name} analysé")

else:
    print("❌ Aucun modèle disponible pour l'analyse")
    print("💡 Vérifiez la connexion internet et réessayez le chargement")

print(f"\n✅ Analyse multi-modèles terminée !")
print(f"📊 {len(comprehensive_results)} cas analysés")
print(f"📚 Prochaine étape : Visualisation comparative avancée")

## 📊 Visualisation Comparative Avancée

Créons des visualisations sophistiquées pour analyser les performances :

In [ ]:
print("📊 VISUALISATION COMPARATIVE AVANCÉE")
print("=" * 40)

def create_advanced_comparative_visualization(comprehensive_results):
    """
    Crée des visualisations avancées pour comparaison multi-modèles
    """
    
    if not comprehensive_results:
        print("❌ Pas de résultats à visualiser")
        return
    
    # 1. Vue d'ensemble : Heatmap des prédictions
    print("\n📈 Création de la heatmap des prédictions...")
    
    # Collecte des données pour la heatmap
    model_names = list(models_dict.keys()) if models_dict else []
    case_names = list(comprehensive_results.keys())
    
    if model_names and case_names:
        # Création de la figure globale
        fig = plt.figure(figsize=(20, 16))
        gs = fig.add_gridspec(4, 3, height_ratios=[1, 1, 1, 0.3], hspace=0.4, wspace=0.3)
        
        # Heatmap globale des détections positives
        ax_heatmap = fig.add_subplot(gs[0, :])
        
        detection_matrix = np.zeros((len(case_names), len(model_names)))
        
        for i, case_name in enumerate(case_names):
            case_results = comprehensive_results[case_name]['model_results']
            for j, model_name in enumerate(model_names):
                if model_name in case_results:
                    positive_count = len(case_results[model_name]['positive_findings'])
                    detection_matrix[i, j] = positive_count
        
        # Création de la heatmap
        im = ax_heatmap.imshow(detection_matrix, cmap='Reds', aspect='auto')
        
        # Configuration des axes
        ax_heatmap.set_xticks(range(len(model_names)))
        ax_heatmap.set_xticklabels([models_info.get(name, {'name': name})['name'] for name in model_names], 
                                  rotation=45, ha='right')
        ax_heatmap.set_yticks(range(len(case_names)))
        ax_heatmap.set_yticklabels([case.replace('_', ' ').title() for case in case_names])
        
        # Ajout des valeurs dans les cellules
        for i in range(len(case_names)):
            for j in range(len(model_names)):
                text = ax_heatmap.text(j, i, int(detection_matrix[i, j]),
                                     ha="center", va="center", color="white", fontweight='bold')
        
        ax_heatmap.set_title("Nombre de Détections Positives par Cas et Modèle", 
                           fontsize=14, fontweight='bold')
        
        # Colorbar
        cbar = plt.colorbar(im, ax=ax_heatmap, orientation='horizontal', pad=0.1)
        cbar.set_label('Nombre de pathologies détectées', fontsize=12)
        
        # 2. Analyse de consensus par cas
        for idx, (case_name, case_data) in enumerate(list(comprehensive_results.items())[:6]):
            row = (idx // 3) + 1
            col = idx % 3
            ax_case = fig.add_subplot(gs[row, col])
            
            # Collecte des probabilités max par modèle
            model_max_probs = []
            model_labels = []
            
            for model_name in model_names:
                if model_name in case_data['model_results']:
                    probs = case_data['model_results'][model_name]['probabilities']
                    max_prob = np.max(probs)
                    model_max_probs.append(max_prob)
                    model_labels.append(models_info.get(model_name, {'name': model_name})['name'][:8])
            
            # Graphique en barres
            colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold'][:len(model_max_probs)]
            bars = ax_case.bar(range(len(model_max_probs)), model_max_probs, 
                              color=colors, alpha=0.7, edgecolor='black')
            
            ax_case.set_xticks(range(len(model_labels)))
            ax_case.set_xticklabels(model_labels, rotation=45, ha='right', fontsize=10)
            ax_case.set_ylabel('Prob. Max', fontsize=10)
            ax_case.set_title(f"{case_name.replace('_', ' ').title()}", fontsize=11, fontweight='bold')
            ax_case.axhline(y=0.5, color='red', linestyle='--', alpha=0.7, linewidth=1)
            ax_case.set_ylim(0, 1)
            ax_case.grid(True, alpha=0.3)
            
            # Valeurs sur les barres
            for bar, prob in zip(bars, model_max_probs):
                height = bar.get_height()
                ax_case.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                           f'{prob:.2f}', ha='center', va='bottom', fontsize=9)
        
        # 3. Légende et statistiques globales
        ax_legend = fig.add_subplot(gs[3, :])
        ax_legend.axis('off')
        
        # Calcul de statistiques globales
        total_analyses = len(case_names) * len(model_names)
        total_detections = np.sum(detection_matrix)
        avg_detections_per_case = np.mean(np.sum(detection_matrix, axis=1))
        avg_detections_per_model = np.mean(np.sum(detection_matrix, axis=0))
        
        stats_text = f"""
📊 STATISTIQUES GLOBALES DE L'ANALYSE COMPARATIVE

• Analyses totales effectuées : {total_analyses}
• Détections positives totales : {int(total_detections)}
• Détections moyennes par cas : {avg_detections_per_case:.1f}
• Détections moyennes par modèle : {avg_detections_per_model:.1f}

📈 INTERPRÉTATION :
• Ligne rouge : Seuil de détection (50%)
• Heatmap : Plus rouge = Plus de détections
• Consensus élevé = Même diagnostic par plusieurs modèles
• Divergences = Incertitudes diagnostiques
        """
        
        ax_legend.text(0.05, 0.95, stats_text, transform=ax_legend.transAxes,
                      fontsize=11, verticalalignment='top', fontfamily='monospace',
                      bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))
        
        plt.suptitle("Analyse Comparative Multi-Modèles TorchXRayVision", 
                    fontsize=18, fontweight='bold', y=0.98)
        plt.tight_layout()
        plt.show()
        
        print("✅ Visualisation comparative créée !")
    else:
        print("❌ Données insuffisantes pour la visualisation")

# Création des visualisations avancées
print("\n🎨 Génération des visualisations avancées...")

if comprehensive_results:
    create_advanced_comparative_visualization(comprehensive_results)
else:
    print("⚠️ Aucun résultat à visualiser")
    print("💡 Assurez-vous que les modèles sont chargés correctement")

print("\n🎯 Visualisation comparative terminée !")

## 📈 Analyse de Performance Détaillée

Analysons en détail les performances de chaque modèle :

In [ ]:
print("📈 ANALYSE DE PERFORMANCE DÉTAILLÉE")
print("=" * 40)

def calculate_model_performance_metrics(comprehensive_results):
    """
    Calcule des métriques de performance détaillées pour chaque modèle
    """
    
    if not comprehensive_results or not models_dict:
        print("❌ Pas de données pour calculer les métriques")
        return {}
    
    print("\n📊 Calcul des métriques de performance...")
    
    model_names = list(models_dict.keys())
    performance_metrics = {}
    
    for model_name in model_names:
        print(f"\n🔍 Analyse du modèle : {models_info.get(model_name, {'name': model_name})['name']}")
        
        # Collecte des données pour ce modèle
        all_probabilities = []
        all_expected_pathologies = []
        all_detected_pathologies = []
        case_performances = []
        
        for case_name, case_data in comprehensive_results.items():
            if model_name in case_data['model_results']:
                model_result = case_data['model_results'][model_name]
                expected = case_data['case_info']['expected_findings']
                detected = [finding[0] for finding in model_result['positive_findings']]
                
                # Stockage pour analyse globale
                all_probabilities.extend(model_result['probabilities'])
                all_expected_pathologies.extend(expected)
                all_detected_pathologies.extend(detected)
                
                # Performance par cas
                case_performance = {
                    'case_name': case_name,
                    'expected': expected,
                    'detected': detected,
                    'max_probability': np.max(model_result['probabilities']),
                    'num_detections': len(detected),
                    'true_positives': 0,
                    'false_positives': len(detected),
                    'false_negatives': len(expected)
                }
                
                # Calcul des vrais positifs (correspondances approximatives)
                for exp_path in expected:
                    for det_path in detected:
                        if (exp_path.lower() in det_path.lower() or 
                            det_path.lower() in exp_path.lower() or
                            any(word in det_path.lower() for word in exp_path.lower().split())):
                            case_performance['true_positives'] += 1
                            case_performance['false_positives'] -= 1
                            case_performance['false_negatives'] -= 1
                            break
                
                case_performances.append(case_performance)
        
        # Calcul des métriques globales
        total_tp = sum(cp['true_positives'] for cp in case_performances)
        total_fp = sum(max(0, cp['false_positives']) for cp in case_performances)
        total_fn = sum(max(0, cp['false_negatives']) for cp in case_performances)
        
        # Métriques calculées
        precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
        recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        # Statistiques des probabilités
        prob_stats = {
            'mean': np.mean(all_probabilities),
            'std': np.std(all_probabilities),
            'max': np.max(all_probabilities),
            'min': np.min(all_probabilities),
            'median': np.median(all_probabilities)
        }
        
        # Stockage des métriques
        performance_metrics[model_name] = {
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'true_positives': total_tp,
            'false_positives': total_fp,
            'false_negatives': total_fn,
            'probability_stats': prob_stats,
            'case_performances': case_performances,
            'total_cases': len(case_performances),
            'avg_detections_per_case': np.mean([len(cp['detected']) for cp in case_performances])
        }
        
        print(f"   ✅ Métriques calculées :")
        print(f"      • Précision : {precision:.3f}")
        print(f"      • Rappel : {recall:.3f}")
        print(f"      • F1-Score : {f1_score:.3f}")
        print(f"      • Détections/cas : {np.mean([len(cp['detected']) for cp in case_performances]):.1f}")
    
    return performance_metrics

def create_performance_comparison_chart(performance_metrics):
    """
    Crée un graphique de comparaison des performances
    """
    if not performance_metrics:
        return
    
    print("\n📊 Création du graphique de comparaison des performances...")
    
    # Données pour les graphiques
    model_names = list(performance_metrics.keys())
    model_labels = [models_info.get(name, {'name': name})['name'] for name in model_names]
    
    precisions = [performance_metrics[name]['precision'] for name in model_names]
    recalls = [performance_metrics[name]['recall'] for name in model_names]
    f1_scores = [performance_metrics[name]['f1_score'] for name in model_names]
    avg_detections = [performance_metrics[name]['avg_detections_per_case'] for name in model_names]
    
    # Création de la figure
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Comparaison des Performances - Modèles TorchXRayVision', 
                fontsize=16, fontweight='bold')
    
    # 1. Précision, Rappel, F1-Score
    ax1 = axes[0, 0]
    x_pos = np.arange(len(model_labels))
    width = 0.25
    
    bars1 = ax1.bar(x_pos - width, precisions, width, label='Précision', 
                    color='lightblue', alpha=0.8, edgecolor='black')
    bars2 = ax1.bar(x_pos, recalls, width, label='Rappel', 
                    color='lightgreen', alpha=0.8, edgecolor='black')
    bars3 = ax1.bar(x_pos + width, f1_scores, width, label='F1-Score', 
                    color='lightcoral', alpha=0.8, edgecolor='black')
    
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(model_labels, rotation=45, ha='right')
    ax1.set_ylabel('Score', fontweight='bold')
    ax1.set_title('Métriques de Classification', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1)
    
    # Ajout des valeurs sur les barres
    for bars in [bars1, bars2, bars3]:
        for bar in bars:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.2f}', ha='center', va='bottom', fontsize=9)
    
    # 2. Détections moyennes par cas
    ax2 = axes[0, 1]
    bars_det = ax2.bar(model_labels, avg_detections, 
                      color='gold', alpha=0.8, edgecolor='black')
    ax2.set_title('Détections Moyennes par Cas', fontweight='bold')
    ax2.set_ylabel('Nombre de Détections', fontweight='bold')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)
    
    for bar in bars_det:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                f'{height:.1f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # 3. Distribution des probabilités
    ax3 = axes[1, 0]
    
    for i, model_name in enumerate(model_names):
        prob_stats = performance_metrics[model_name]['probability_stats']
        
        # Création d'un violinplot simplifié
        model_label = model_labels[i]
        mean_prob = prob_stats['mean']
        std_prob = prob_stats['std']
        
        ax3.bar(i, mean_prob, yerr=std_prob, capsize=5, 
               color=f'C{i}', alpha=0.7, edgecolor='black',
               label=f'{model_label}: μ={mean_prob:.3f}')
    
    ax3.set_xticks(range(len(model_labels)))
    ax3.set_xticklabels(model_labels, rotation=45, ha='right')
    ax3.set_ylabel('Probabilité Moyenne', fontweight='bold')
    ax3.set_title('Distribution des Probabilités', fontweight='bold')
    ax3.axhline(y=0.5, color='red', linestyle='--', alpha=0.7, label='Seuil 50%')
    ax3.grid(True, alpha=0.3)
    ax3.legend()
    
    # 4. Matrice de confusion agrégée
    ax4 = axes[1, 1]
    
    # Calcul de la matrice de confusion moyennée
    confusion_data = []
    confusion_labels = []
    
    for model_name in model_names:
        metrics = performance_metrics[model_name]
        tp = metrics['true_positives']
        fp = metrics['false_positives']
        fn = metrics['false_negatives']
        
        confusion_data.append([tp, fp, fn])
        confusion_labels.append(models_info.get(model_name, {'name': model_name})['name'])
    
    confusion_array = np.array(confusion_data)
    
    # Heatmap de la confusion
    im = ax4.imshow(confusion_array.T, cmap='Blues', aspect='auto')
    
    ax4.set_xticks(range(len(confusion_labels)))
    ax4.set_xticklabels(confusion_labels, rotation=45, ha='right')
    ax4.set_yticks(range(3))
    ax4.set_yticklabels(['Vrais Positifs', 'Faux Positifs', 'Faux Négatifs'])
    ax4.set_title('Matrice de Confusion Agrégée', fontweight='bold')
    
    # Ajout des valeurs
    for i in range(len(confusion_labels)):
        for j in range(3):
            text = ax4.text(i, j, int(confusion_array[i, j]),
                           ha="center", va="center", color="white", fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Graphiques de comparaison créés !")

# Calcul et visualisation des performances
if comprehensive_results:
    print("\n🧮 Calcul des métriques de performance...")
    performance_metrics = calculate_model_performance_metrics(comprehensive_results)
    
    if performance_metrics:
        create_performance_comparison_chart(performance_metrics)
        
        # Résumé textuel des performances
        print(f"\n📋 RÉSUMÉ COMPARATIF DES PERFORMANCES")
        print(f"=" * 50)
        
        best_precision = max(performance_metrics.items(), key=lambda x: x[1]['precision'])
        best_recall = max(performance_metrics.items(), key=lambda x: x[1]['recall'])
        best_f1 = max(performance_metrics.items(), key=lambda x: x[1]['f1_score'])
        
        print(f"🥇 Meilleure précision : {models_info.get(best_precision[0], {'name': best_precision[0]})['name']} ({best_precision[1]['precision']:.3f})")
        print(f"🥇 Meilleur rappel : {models_info.get(best_recall[0], {'name': best_recall[0]})['name']} ({best_recall[1]['recall']:.3f})")
        print(f"🥇 Meilleur F1-Score : {models_info.get(best_f1[0], {'name': best_f1[0]})['name']} ({best_f1[1]['f1_score']:.3f})")
        
        print(f"\n💡 Recommandations cliniques :")
        print(f"• Modèle le plus précis pour éviter faux positifs : {models_info.get(best_precision[0], {'name': best_precision[0]})['name']}")
        print(f"• Modèle le plus sensible pour dépistage : {models_info.get(best_recall[0], {'name': best_recall[0]})['name']}")
        print(f"• Modèle le plus équilibré : {models_info.get(best_f1[0], {'name': best_f1[0]})['name']}")
    
else:
    print("⚠️ Aucune donnée de performance disponible")

print("\n🎯 Analyse de performance détaillée terminée !")

## 💾 Sauvegarde et Rapport Complet

In [ ]:
print("💾 GÉNÉRATION DU RAPPORT COMPLET")
print("=" * 35)

def generate_comprehensive_report(comprehensive_results, performance_metrics, session_dir):
    """
    Génère un rapport complet de l'analyse multi-modèles
    """
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    print(f"\n📝 Génération du rapport complet...")
    
    # Rapport textuel détaillé
    report_path = f"{session_dir}rapport_comparatif_{timestamp}.txt"
    
    report_content = f"""
RAPPORT COMPARATIF MULTI-MODÈLES TORCHXRAYVISION
===============================================

Date d'analyse : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Tutorial : 2 - Classification Avancée
Version TorchXRayVision : {xrv.__version__}
Dispositif d'analyse : {device}

MODÈLES ANALYSÉS :
=================
"""
    
    for model_name, info in models_info.items():
        if model_name in models_dict:
            report_content += f"""
• {info['name']} :
  - Architecture : {info['architecture']}
  - Données d'entraînement : {info['training_data']}
  - Spécialité : {info['specialty']}
  - Paramètres : {info['params']:,}
"""
    
    report_content += f"""

CAS CLINIQUES ANALYSÉS :
=======================
"""
    
    for case_name, case_data in clinical_cases.items():
        expected = case_data['expected_findings']
        context = case_data['clinical_context']
        description = case_data['description']
        
        report_content += f"""
• {case_name.upper()} :
  - Description : {description}
  - Pathologies attendues : {', '.join(expected)}
  - Contexte clinique : {context}
"""
    
    if performance_metrics:
        report_content += f"""

PERFORMANCES COMPARATIVES :
==========================
"""
        
        for model_name, metrics in performance_metrics.items():
            model_info = models_info.get(model_name, {'name': model_name})
            report_content += f"""
• {model_info['name']} :
  - Précision : {metrics['precision']:.3f}
  - Rappel (Sensibilité) : {metrics['recall']:.3f}
  - F1-Score : {metrics['f1_score']:.3f}
  - Vrais positifs : {metrics['true_positives']}
  - Faux positifs : {metrics['false_positives']}
  - Faux négatifs : {metrics['false_negatives']}
  - Détections/cas : {metrics['avg_detections_per_case']:.1f}
  - Probabilité moyenne : {metrics['probability_stats']['mean']:.3f}
"""
    
    report_content += f"""

ANALYSE DÉTAILLÉE PAR CAS :
===========================
"""
    
    for case_name, case_result in comprehensive_results.items():
        case_info = case_result['case_info']
        model_results = case_result['model_results']
        
        report_content += f"""
CAS : {case_name.upper()}
Description : {case_info['description']}
Contexte : {case_info['clinical_context']}
Pathologies attendues : {', '.join(case_info['expected_findings'])}

Résultats par modèle :
"""
        
        for model_name, result in model_results.items():
            model_info = models_info.get(model_name, {'name': model_name})
            positive_findings = result['positive_findings']
            max_prob = np.max(result['probabilities'])
            
            report_content += f"""
  • {model_info['name']} :
    - Probabilité maximale : {max_prob:.3f}
    - Détections positives : {len(positive_findings)}
"""
            
            if positive_findings:
                report_content += "    - Pathologies détectées :\n"
                for pathology, prob in sorted(positive_findings, key=lambda x: x[1], reverse=True):
                    report_content += f"      • {pathology}: {prob:.3f}\n"
            else:
                report_content += "    - Aucune pathologie détectée (seuil 50%)\n"
    
    # Conclusions et recommandations
    if performance_metrics:
        best_models = {
            'precision': max(performance_metrics.items(), key=lambda x: x[1]['precision']),
            'recall': max(performance_metrics.items(), key=lambda x: x[1]['recall']),
            'f1': max(performance_metrics.items(), key=lambda x: x[1]['f1_score'])
        }
        
        report_content += f"""

CONCLUSIONS ET RECOMMANDATIONS CLINIQUES :
==========================================

PERFORMANCES OPTIMALES :
• Meilleure précision : {models_info.get(best_models['precision'][0], {'name': best_models['precision'][0]})['name']} ({best_models['precision'][1]['precision']:.3f})
• Meilleure sensibilité : {models_info.get(best_models['recall'][0], {'name': best_models['recall'][0]})['name']} ({best_models['recall'][1]['recall']:.3f})
• Meilleur équilibre : {models_info.get(best_models['f1'][0], {'name': best_models['f1'][0]})['name']} ({best_models['f1'][1]['f1_score']:.3f})

RECOMMANDATIONS D'USAGE CLINIQUE :

1. DÉPISTAGE (priorité à la sensibilité) :
   → Utiliser {models_info.get(best_models['recall'][0], {'name': best_models['recall'][0]})['name']}
   → Minimise les faux négatifs
   → Adapté pour screening de masse

2. DIAGNOSTIC DE CONFIRMATION (priorité à la précision) :
   → Utiliser {models_info.get(best_models['precision'][0], {'name': best_models['precision'][0]})['name']}
   → Minimise les faux positifs
   → Adapté pour éviter sur-traitement

3. USAGE GÉNÉRAL (équilibre optimal) :
   → Utiliser {models_info.get(best_models['f1'][0], {'name': best_models['f1'][0]})['name']}
   → Bon compromis sensibilité/spécificité
   → Adapté pour consultation courante

LIMITES ET CONSIDÉRATIONS :

• Validation sur données synthétiques - Performance réelle peut différer
• Variabilité inter-observateur non prise en compte
• Nécessité de validation sur cohortes cliniques réelles
• L'IA reste un outil d'aide - Décision finale au médecin
• Formation continue nécessaire sur ces technologies

PERSPECTIVES D'AMÉLIORATION :

• Ensembles de modèles pour robustesse accrue
• Spécialisation par type de pathologie
• Intégration de données cliniques contextuelles
• Développement de modèles explicables (XAI)

---
Rapport généré par TorchXRayVision Tutorial 2
Classification Avancée pour Formation Médicale
"""
    
    # Sauvegarde du rapport
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(report_content)
    
    print(f"   ✅ Rapport textuel : {report_path}")
    
    # Sauvegarde des données JSON
    json_path = f"{session_dir}donnees_comparatives_{timestamp}.json"
    
    json_data = {
        'timestamp': timestamp,
        'models_info': models_info,
        'performance_metrics': performance_metrics if performance_metrics else {},
        'clinical_cases': {name: {'description': info['description'], 
                                 'expected_findings': info['expected_findings']}
                          for name, info in clinical_cases.items()}
    }
    
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)
    
    print(f"   ✅ Données JSON : {json_path}")
    
    return report_path, json_path

# Génération du rapport final
if comprehensive_results:
    print("\n📊 Finalisation de l'analyse comparative...")
    
    report_files = generate_comprehensive_report(
        comprehensive_results, 
        performance_metrics if 'performance_metrics' in locals() else {},
        session_dir
    )
    
    print(f"\n✅ Rapport complet généré !")
    print(f"\n📁 Fichiers sauvegardés dans : {session_dir}")
    print(f"📄 Types de fichiers créés :")
    print(f"   • Rapport textuel détaillé")
    print(f"   • Données JSON pour réutilisation")
    print(f"   • Visualisations PNG (si générées)")
    
else:
    print("⚠️ Aucun résultat à sauvegarder")

print(f"\n🎯 Tutorial 2 - Classification Avancée Terminé !")

## 🎉 Conclusion du Tutorial 2

### 🏆 Félicitations !

Vous maîtrisez maintenant la **classification avancée multi-modèles** avec TorchXRayVision !

### ✅ Compétences Acquises :

#### **Techniques Avancées :**
1. **🧠 Chargement multi-modèles** : DenseNet121 sur différents datasets
2. **🖼️ Création de cas complexes** : Simulation de pathologies variées
3. **🔬 Analyse comparative** : Performance croisée des modèles
4. **📊 Métriques avancées** : Précision, Rappel, F1-Score
5. **📈 Visualisations sophistiquées** : Heatmaps, graphiques comparatifs
6. **📋 Rapports automatisés** : Documentation complète

#### **Cliniques :**
- **Choix de modèle** selon contexte clinique
- **Interprétation multi-pathologies**
- **Gestion de l'incertitude** diagnostique
- **Recommandations thérapeutiques** basées sur l'IA

### 📊 Résultats de Performance

Vous avez appris à évaluer :
- **Sensibilité vs Spécificité** selon l'usage clinique
- **Consensus entre modèles** pour robustesse
- **Trade-offs cliniques** entre faux positifs/négatifs
- **Adaptation contextuelle** des seuils de détection

### 🏥 Applications Cliniques Maîtrisées

#### **Dépistage de Masse :**
- Priorité à la **sensibilité élevée**
- Modèles optimisés pour **détection précoce**
- Workflow de **triage automatique**

#### **Diagnostic de Confirmation :**
- Priorité à la **spécificité élevée**
- Évitement des **sur-diagnostics**
- **Précision maximale** requise

#### **Consultation Générale :**
- **Équilibre optimal** sensibilité/spécificité
- **Polyvalence** diagnostique
- **Facilité d'interprétation**

### 🎯 Prochaines Étapes

#### **Tutorial 3** : Segmentation Anatomique
- Délimitation automatique des structures
- Mesures morphométriques précises
- Analyse quantitative des organes
- Applications en radiothérapie

#### **Tutorial 4** : Détection et Localisation
- Localisation précise des pathologies
- Cartes d'activation (Grad-CAM)
- Explicabilité de l'IA (XAI)
- Confiance et incertitude

#### **Tutorial 5** : Comparaison Multi-Architectures
- ResNet vs DenseNet vs Vision Transformers
- Architectures émergentes
- Optimisation des performances
- Benchmarking complet

### 💡 Points Clés à Retenir

#### **Choix du Modèle :**
- **Dataset d'entraînement** détermine la spécialisation
- **Taille du modèle** vs **vitesse d'inférence**
- **Contexte clinique** guide la sélection

#### **Interprétation des Résultats :**
- **Probabilités ≠ Certitudes** diagnostiques
- **Consensus multi-modèles** = Robustesse accrue
- **Corrélation clinique** toujours nécessaire

#### **Limitations à Considérer :**
- **Biais des datasets** d'entraînement
- **Variabilité des présentations** cliniques
- **Évolution technologique** rapide

### 🌟 Message Final

La **classification multi-modèles** représente l'état actuel de l'art en IA radiologique. Vous maîtrisez maintenant les outils qui transforment la pratique médicale mondiale.

#### **Votre Expertise :**
- **Analyse comparative** rigoureuse
- **Choix éclairés** de modèles
- **Interprétation clinique** avancée
- **Documentation professionnelle**

**Prêt pour le Tutorial 3 ?** La segmentation anatomique vous attend ! 🔬✨

---

*Vous maîtrisez désormais les fondamentaux de la classification IA en radiologie !*